<a href="https://colab.research.google.com/github/luiza457/fer-thesis/blob/main/FERModel_CNN_NoAttention_Updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Firstly, we are going to build the CNN pipeline for local facial features.
Step 1. Train the model
Step 2. Add local attention inside CNN layers

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
import torch.optim as optim
import kagglehub
import os
from tqdm.notebook import tqdm


# Download latest version
path = kagglehub.dataset_download("thienkhonghoc/affectnet")

# print("Path to dataset files:", path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


class CNNFERModel(nn.Module):
    def __init__(self, num_classes=8):
        super(CNNFERModel, self).__init__()

        # load the pre-trained ResNet18 model
        # 18 is faster & easier to debug, use 50 later for better accuracy
        base_model = models.resnet18(weights=True)

        # remove last layer (FC)
        # we need to output features, not scores as predictions
        self.backbone = nn.Sequential(*list(base_model.children())[:-1])

        # custom classification head
        self.classifier = nn.Sequential(
            # reduce output features to 256
            nn.Linear(base_model.fc.in_features, 256),
            # add non-linearity to learn complex patterns
            nn.ReLU(),
            # turn off 30% neurons to prevent overfitting (regularization)
            nn.Dropout(0.3),
            # map the features to the 7 emotion classes
            nn.Linear(256, num_classes)
        )

    # feed in an img => most important features
    # flatten them => returns logits
    def forward(self, x):
        x = self.backbone(x)       # [B, 512, 1, 1]
        x = x.view(x.size(0), -1)  # Flatten
        out = self.classifier(x)   # [B, num_classes]
        return out

# load dataset
train_dir = "~/.cache/kagglehub/datasets/thienkhonghoc/affectnet/versions/3/AffectNet/train"
valid_dir =  "~/.cache/kagglehub/datasets/thienkhonghoc/affectnet/versions/3/AffectNet/val"
test_dir =  "~/.cache/kagglehub/datasets/thienkhonghoc/affectnet/versions/3/AffectNet/test"

train_transform = transforms.Compose(
    [
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(degrees=15),
     transforms.ColorJitter(brightness=0.3, contrast=0.3),
     transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

valid_transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

test_transform = transforms.Compose(
    [
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

batch_size = 32

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
valid_dataset = datasets.ImageFolder(root=valid_dir, transform=valid_transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=test_transform)

trainloader = DataLoader(train_dataset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

validloader = DataLoader(valid_dataset, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

testloader = DataLoader(test_dataset, batch_size=batch_size,
                                          shuffle=False, num_workers=0)

print(f"Length of train_dataset: {len(train_dataset)}")
print(f"Length of trainloader: {len(trainloader)}")
# create model
fermodel = CNNFERModel()
fermodel.to(device)

criterion = nn.CrossEntropyLoss()

# use the adam optimizer
optimizer = optim.Adam(fermodel.parameters(), lr=0.001)



# training loop
# training loop
best_valid_acc = 0.0
try:
  for epoch in range(25):
      fermodel.train()  # Set model to training mode
      running_loss = 0.0
      train_correct, train_total = 0, 0

      progress_bar = tqdm(trainloader, desc=f"Epoch {epoch+1}/{25}", leave=False,mininterval=0.5, miniters=1)

      for i, data in enumerate(trainloader):
          inputs, labels = data
          inputs = inputs.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()

          outputs = fermodel(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          # update weights
          optimizer.step()

          running_loss += loss.item()
          _, preds = torch.max(outputs, 1)
          train_correct += (preds == labels).sum().item()
          train_total += labels.size(0)

          progress_bar.update(1)


      train_acc = 100 * train_correct / train_total
      train_loss = running_loss / len(trainloader)

      # validation loop
      fermodel.eval()  # Set model to evaluation mode
      val_loss = 0.0
      val_correct, val_total = 0, 0

      with torch.no_grad():
          for val_inputs, val_labels in validloader:
              val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)

              val_outputs = fermodel(val_inputs)
              val_loss_batch = criterion(val_outputs, val_labels)
              val_loss += val_loss_batch.item()

              _, predicted = torch.max(val_outputs.data, 1)
              val_total += val_labels.size(0)
              val_correct += (predicted == val_labels).sum().item()

      avg_val_loss = val_loss / len(validloader)
      val_accuracy = 100 * val_correct / val_total

      # Update tqdm bar with live stats
      progress_bar.set_postfix({
          'Train Loss': f'{train_loss:.4f}',
          'Train Acc': f'{train_acc:.2f}%'
      })
      progress_bar.close()


      print(f"Epoch {epoch+1}/{25} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | Val Loss: {avg_val_loss:.4f} | Val Acc: {val_accuracy:.2f}%")


      # save model if validation accuracy is improved
      if val_accuracy > best_valid_acc:
          best_valid_acc = val_accuracy
          torch.save(fermodel.state_dict(), 'cnnmodel_noattention.pth')
          print(f"Model saved at epoch {epoch+1} with validation accuracy: {best_valid_acc:.2f}%")

except Exception as e:
    print(f"An error occurred: {e}")
    import traceback
    traceback.print_exc()
print('Finished Training')


Length of train_dataset: 37553
Length of trainloader: 1174


/opt/conda/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Epoch 1/25:   0%|          | 0/1174 [00:00<?, ?it/s]

Epoch 1/25 | Train Loss: 1.6165 | Train Acc: 37.78% | Val Loss: 1.6158 | Val Acc: 39.75%
Model saved at epoch 1 with validation accuracy: 39.75%


Epoch 2/25:   0%|          | 0/1174 [00:00<?, ?it/s]

...

Epoch 23/25 | Train Loss: 0.7968 | Train Acc: 70.52% | Val Loss: 1.4937 | Val Acc: 54.50%


Epoch 24/25:   0%|          | 0/1174 [00:00<?, ?it/s]

Epoch 24/25 | Train Loss: 0.7679 | Train Acc: 71.75% | Val Loss: 1.4318 | Val Acc: 54.25%


Epoch 25/25:   0%|          | 0/1174 [00:00<?, ?it/s]

Epoch 25/25 | Train Loss: 0.7410 | Train Acc: 72.64% | Val Loss: 1.4326 | Val Acc: 57.00%
Finished Training


In [ ]:
import torch
from google.colab import drive
drive.mount('/content/drive')

model_path = '/content/drive/MyDrive/cnnmodel_noattention.pth'
model = torch.load(model_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
